In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [162]:
user='G'

if user=='S':
    df=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/liste_mandat.csv")#on récupère le csv de la liste des mandats
    df_depu=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/listeDepute_complete.csv")

if user=='G':
    df=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/liste_mandatdef.csv") 
    df_depu=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/listeDepute_complete.csv")


In [163]:
debut_mandat=[]
for dates in df["Mandat"]:
    dates=str(dates).split()
    debut_mandat.append(' '.join(dates[1:4]))
    
df["Debut Mandat"]=np.array(debut_mandat)
df.head()

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Nom,Debut Mandat
0,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,Damien Meslot,19 juin 2002
1,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,Damien Meslot,20 juin 2007
2,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIVe législature,Du 20 juin 2012 au 20 juin 2017,Cinquième République - Assemblée nationale,Damien Meslot,20 juin 2012
3,0,Ille-et-Vilaine,La République en Marche,XVe législature (Législature en cours),Depuis le 21 juin 2017 (mandat en cours),Cinquième République - Assemblée nationale,Christine Cloarec-Le Nabour,le 21 juin
4,0,Ardèche,Socialiste,XIe législature,Du 1er juin 1997 au 18 juin 2002,Cinquième République - Assemblée nationale,Pascal Terrasse,1er juin 1997


In [164]:
mandat_dep=df.sort_values(by='Département')
mandat_dep.head()

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Nom,Debut Mandat
28119,0,Ain,Majorité ministérielle,IVe législature,Du 24 novembre 1827 au 16 mai 1830,Seconde Restauration - Chambre des députés des...,"Jean-Baptiste, Antoine Georgette du Buisson de...",24 novembre 1827
7717,0,Ain,Mouvement républicain populaire,Ire Assemblée nationale constituante,Du 21 octobre 1945 au 10 juin 1946,Gouvernement provisoire de la République franç...,"Pierre, Hector Dominjon",21 octobre 1945
5277,0,Ain,Socialiste,VIIe législature,Du 2 juillet 1981 au 1er avril 1986,Cinquième République - Assemblée nationale,Louis Robin,2 juillet 1981
30945,0,Ain,Droite,IIIe législature,Du 6 mars 1824 au 5 novembre 1827,Seconde Restauration - Chambre des députés des...,"Claude, Marie, Joseph Dumarché-Bolozon",6 mars 1824
30944,0,Ain,Droite,IIe législature,Du 20 novembre 1822 au 24 décembre 1823,Seconde Restauration - Chambre des députés des...,"Claude, Marie, Joseph Dumarché-Bolozon",20 novembre 1822


In [165]:
df_depu.head()

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort
0,0,Ludovic Pajot,18 novembre 1993,NaN,NaN
1,1,Typhanie Degois,6 janvier 1993,NaN,NaN
2,2,Lénaïck Adam,19 février 1992,NaN,NaN
3,3,Pierre Henriet,26 novembre 1991,NaN,NaN
4,4,Robin Reda,10 mai 1991,NaN,NaN


<b> TRANSTYPAGE !!! (Je suis heureux que ça ait marché) 

In [178]:
df_depu[["Nom","Né le"]].head(2)
mandat_dep[["Nom","Debut Mandat"]].head(2)
age_dep=mandat_dep.set_index('Nom').join(df_depu[["Nom","Né le"]].set_index('Nom'), on='Nom')
dates=[]
mandat_dep.reset_index()
mois={'janvier':1,'février':2, 'mars':3, 'avril':4, 'mai':5, 'juin':6, 'juillet':7, 'août':8, 'septembre':9, 'octobre':10, 'novembre':11, 'décembre':12}
def transtype_date(x):
    x=str(x)        
    date_texte=x.split() 
    if len(date_texte)>1 :
        if  date_texte[1] in (mois.keys()):
            if date_texte[0]=="1er":
                date_texte[0]='1'
            date_texte[1]=str(mois[date_texte[1]])
            date_nombre='/'.join(date_texte)
            return pd.to_datetime(date_nombre)
    else:
        return pd.NaT

def nul(x):
    return 0
age_dep["Date de naissance"]=age_dep["Né le"].apply(lambda x: transtype_date(x))
age_dep["Date debut mandat"]=age_dep["Debut Mandat"].apply(lambda x: transtype_date(x))
age_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Date de naissance,Date debut mandat
Nom,,,,,,,,,,
"Jean-Baptiste, Antoine Georgette du Buisson de La Boulaye",0,Ain,Majorité ministérielle,IVe législature,Du 24 novembre 1827 au 16 mai 1830,Seconde Restauration - Chambre des députés des...,24 novembre 1827,12 novembre 1781,1781-12-11,1827-11-24
"Pierre, Hector Dominjon",0,Ain,Mouvement républicain populaire,Ire Assemblée nationale constituante,Du 21 octobre 1945 au 10 juin 1946,Gouvernement provisoire de la République franç...,21 octobre 1945,8 octobre 1910,1910-08-10,1945-10-21


In [191]:
#A convertir en année
(age_dep["Date debut mandat"][1]-age_dep["Date de naissance"][1])

Timedelta('12856 days 00:00:00')

In [99]:
age_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Debut Mandat date
Nom,,,,,,,,,
"Jean-Baptiste, Antoine Georgette du Buisson de La Boulaye",0,Ain,Majorité ministérielle,IVe législature,Du 24 novembre 1827 au 16 mai 1830,Seconde Restauration - Chambre des députés des...,24 novembre 1827,12 novembre 1781,24/11/1827
"Pierre, Hector Dominjon",0,Ain,Mouvement républicain populaire,Ire Assemblée nationale constituante,Du 21 octobre 1945 au 10 juin 1946,Gouvernement provisoire de la République franç...,21 octobre 1945,8 octobre 1910,21/10/1945


In [50]:
mandat_dep['âge député']=mandat_dep[df_depu['Né le'][df_depu['nom'==mandat_dep[mandat_dep['nom']]]]-mandat_dep['date']]

KeyError: 'date naissance'

In [ ]:
dep_possible=list(mandat_dep['Département'].unique())#une liste de tous les départements mentionnés
dep_possible.remove(" Département d'élection inconnu")
for dep in dep_possible :#on enlève les nan
    if type(dep)!=str :
        dep_possible.remove(dep)
#dep_possible#attention, il y a souvent un espace avant !


In [ ]:
#Le résultat est équivalent comme ça
dep_possible=list(mandat_dep['Département'].unique())[:-1]#une liste de tous les départements mentionnés
dep_possible.remove(" Département d'élection inconnu")
dep_possible

In [ ]:
mandat_dep.count()#on vérifie qu'il n'y a pas trop de manques

In [ ]:
mandat_dep.set_index('Département', inplace=True)#on met département en index pour pouvoir scinder par départements

In [ ]:
liste_dep={}#on crée un dictionnaire où on va mettre les dataframes par départements
for dep in dep_possible:#pour chaque département
    mand_du_dep=mandat_dep.loc[dep]#on récupère le dataframe des mandats dans le département     
    liste_dep[dep]=mand_du_dep#et on l'ajoute au dictionnaire


In [ ]:
carac_dep={}
for dep in dep_possible :
    nbre_mandat=liste_dep[dep]['nom du député'].count()#on compte le nombre total de mandat dans le département
    cpte_depute=0#on crée la variable qui va compter le nombre de députés différents
    compte=[]#on crée une liste où on va mettre les députés déjà comptés
    for depute in liste_dep[dep]['nom du député'] :#on boucle sur les noms de députés
        if depute not in compte :#si on ne l'a pas déjà compté
            cpte_depute+=1#on rajoute un au compte
            compte.append(depute)#et on indique qu'on l'a désormais compté
    carac_dep[dep]=pd.DataFrame(['nombre_mandat', 'nombre député','nombre moyen de mandats par député'], [nbre_mandat, cpte_depute, nbre_mandat/cpte_depute])#on crée un dataframe avec les informations
    



In [ ]:
liste_dep

In [ ]:
#Voila un code plus court
mandat_dep.reset_index()
x,y,z=[],[],[]
for dep in dep_possible :
    x+=[len(df[df['Département']==dep])]#on compte le nombre total de mandat dans le département
    y+=[len(df[df['Département']==dep]['Nom'].unique())]#Puis le nombre de députés différent par dpt
    z+=[x[-1]/y[-1]] #Ceci est le nombre moyen de mandat par député
    
carac_dep_bis=np.array([x, y, z])#on crée un dataframe avec les informations
table = pd.DataFrame(carac_dep_bis, index=['nombre_mandat', 'nombre député','nombre moyen de mandats par député'],columns=dep_possible)
table.head()

